In [2]:
## Libraries
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

## Plot settings
plt.rc('xtick', direction='in', labelsize=14)
plt.rc('ytick', direction='in', labelsize=14)
plt.rc('axes', labelsize=20, titlesize=22)
plt.rc('legend', fontsize=14)
plt.rc('animation', html='jshtml')
plt.rcParams['font.family'] = 'DeJavu Serif'
plt.rcParams['font.serif'] = ['Computer Modern Roman']
plt.rcParams["font.size"] = "12"
plt.rcParams['mathtext.fontset'] = 'cm'

In [3]:
## Read coordinates and indices

grid_coordinates = []
with open('../../Grids/Pe_grid.txt', 'r') as reader:
    line = reader.readline()

    while line != '':
        elements = line.split(',')
        point = (float(elements[0]),float(elements[1]))
        grid_coordinates.append(point)
        line = reader.readline()

jump_coordinates = []
with open('../../Grids/Pe_jump_coordinates.txt', 'r') as reader:
    line = reader.readline()

    while line != '':
        elements = line.split(' ')
        data_block = []
        for ele in elements:
            temp = ele.replace('(', '').replace(')', '')
            if temp != 'stop':
                x = temp.split(',')[0]
                y = temp.split(',')[1]
                point = (float(x),float(y))
                data_block.append(point)
            else:
                break
        line = reader.readline()
        jump_coordinates.append(data_block)

jump_indices = []
with open('../../Grids/Pe_jump_indices.txt', 'r') as reader:
    line = reader.readline()

    while line != '':
        elements = line.split(' ')
        data_block = []
        for ele in elements:
            if ele != 'stop':
                data_block.append(int(ele))
            else:
                break
        line = reader.readline()
        jump_indices.append(data_block)

bott_indices = []
with open('Pe_Bott_indices_B_scale_0100.txt', 'r') as reader:
    line = reader.readline()
    
    while line != '':
        ele = line.replace('(', '').replace(')', '')
        elements = ele.split(',')
        package = (float(elements[0]), int(elements[1]))
        line = reader.readline()
        bott_indices.append(package)

# For drawing vertices use grid_coordinates
# For drawing lines use jump_coordinates
# For indices use jump_indices

## We now want to scale down the plot so it will fit inside a unit square in the 1st quadrant

# Compare x and y components seperatly to for the maximal distances along each axis
xs = [x for x,y in grid_coordinates]
ys = [y for x,y in grid_coordinates]

xs_length = abs(max(xs)) + abs(min(xs))
ys_length = abs(max(ys)) + abs(min(ys))

# We can now scale all xs and ys to fit inside a unit square
xs_scaled = [i/xs_length for i in xs]
ys_scaled = [i/ys_length for i in ys]
grid_scaled = [(x,y) for x, y in zip(xs_scaled, ys_scaled)]

# Same thing for all jump coodinates
jump_scaled = []
for i in jump_coordinates:
    xs_temp = [j[0]/xs_length for j in i]
    ys_temp = [j[1]/ys_length for j in i]
    jump_temp = [(x,y) for x, y in zip(xs_temp, ys_temp)]
    jump_scaled.append(jump_temp)


# The new vertices are stored in grid_scaled
# The new jumps are stored in jump_scaled

## IMPORTANT WE WILL USE THE GRID SCALED FOR ROTATIONS THIS MAKES THE WORK EASIER ##

## DEFINITION OF USEFUL FUNCTIONS

# We now change the Hamiltonian to be rotationally symmetric a thus we change the Peierls phase
# Calculation in notes
# Everything else is identical to the other rotation script
def Hamil_solve(z, w, B):
    # Define now a skeleton of the Hamiltonian 
    dim_H = len(z)
    H = np.zeros((dim_H, dim_H), dtype= np.complex128)
    k_H = [x for x in range(0, dim_H)]
    # Calculate Peierls phase factor according to the 1st artcle 
    
    # Start by finding the tile length
    tile_lens = [np.linalg.norm(np.array(z[0]) - np.array(x)) for x in w[0]]
    tile_lens_sum = np.sum(tile_lens)
    l  = tile_lens_sum/len(tile_lens)
    phi = B * l**2
    phi_0 = 2 * np.pi
    jump_products = []
    for i, j in zip(z, w):
        products = [(i[0]*k[1] - k[0]*i[1]) for k in j]
        jump_products.append(products)
    phase_factors = []
    for i, j in zip(z, jump_products):
        phases = [-(phi/(2 * l**2)) * k for k in j]
        phase_factors.append(phases)

    # Scaling via J, maybe energy factor
    J = 1
    # Fill out the Hamiltonian according to the psi vector, i.e. the basis chosen through generation of the tiling
    for x, y, z in zip(k_H, jump_indices, phase_factors):
        for k, t in zip(y, z):
            H[x, k] = -J * np.exp(1j * t)

    # Now we find the eigenvalues and eigenvectors
    # Note that eigh returns normalized eigenvectors!
    # Note the eigenvectors are arranged in a matrix so eig_vecs[:,i] chooses the i'th column which is the i'th eigenvector
    eig_vals, U = np.linalg.eigh(H)
    eig_vecs = np.array([U[:,i] for i in range(len(U))])
    return eig_vals, eig_vecs, H

# We want to know when phi/phi0 = 1, this happens when phi = 2pi
# phi in this case is phi = B*l*2 so B must be equal to
B_full = 812.549720608777
B_scaling = 0.1
# Take field strength of B= 2pi so that phi/phi0= 0.5
evals, evecs, H = Hamil_solve(grid_scaled, jump_scaled, (B_scaling)*B_full)

In [4]:
bott_indices

[(-1.0000000000000013, 22),
 (-0.9999999999999986, 23),
 (-1.0000000000000004, 24),
 (-1.0000000000000007, 25),
 (-0.9999999999999998, 26),
 (-0.9999999999999983, 27),
 (-0.9999999999999983, 28),
 (-0.9999999999999983, 48),
 (-1.0000000000000007, 49),
 (-1.0000000000000002, 50),
 (-1.0000000000000002, 51),
 (-1.0000000000000029, 52),
 (-0.9999999999999992, 53),
 (-1.0000000000000016, 54),
 (-0.9999999999999978, 61),
 (-1.0000000000000007, 69),
 (-0.9999999999999994, 70),
 (-1.0000000000000027, 71),
 (-1.0000000000000027, 72),
 (-0.999999999999997, 73),
 (-1.0000000000000024, 74),
 (-1.0000000000000018, 75),
 (-0.9999999999999989, 76),
 (-1.0000000000000029, 77),
 (-1.0000000000000013, 78),
 (-1.0000000000000029, 79),
 (-1.0000000000000027, 80),
 (-0.9999999999999993, 81),
 (-1.000000000000002, 82),
 (-0.999999999999998, 83),
 (-1.0000000000000009, 84),
 (-1.0000000000000027, 85),
 (-1.0000000000000013, 86),
 (-0.9999999999999989, 87),
 (-1.0000000000000009, 88),
 (-0.9999999999999967, 

In [8]:
# Now we look to plot DOS for the different eigenvectors

BIs_index_1 = [x for y,x in bott_indices if abs(y) >= 0.5 and abs(y) <= 1.5]
BIs_index_2 = [x for y,x in bott_indices if abs(y) >= 1.5]

relevant_evecs_BI1 = [evecs[i] for i in BIs_index_1]
relevant_evecs_BI2 = [evecs[i] for i in BIs_index_2]

for index, phi in zip(BIs_index_2, relevant_evecs_BI2):
    densities = [(x.conj() * x).real for x in phi]

    # Define figure
    fig, ax = plt.subplots(figsize=(8,8))

    # Plot all possible paths between vertices
    # Plot all possible paths between vertices
    for i,j in zip(grid_scaled, jump_scaled):
        xs = [w[0] for w in j]
        ys = [w[1] for w in j]
        for l,k in zip(xs,ys):
            ax.plot([i[0], l], [i[1], k], color= 'k', zorder= 1, linewidth= 1)

    marker_size = 30

    xs = [w[0] for w in grid_scaled]
    ys = [w[1] for w in grid_scaled]

    # Set colour gradient
    plt.set_cmap('hot_r')

    # Plot, noting c sets the density
    cax = ax.scatter(x= xs, y= ys, s= marker_size, c= densities, zorder= 3, facecolor= 'r')
    # Attach colorbar
    fig.colorbar(cax)

    # Finalize
    plt.axis('equal')
    plt.title(r'State = {}, $\phi / \phi_0$ = {}'.format(index, B_scaling))

    plt.savefig(f'Pe_0100_BI_states/Pe_BI_2/index_{index}', bbox_inches= 'tight')
    plt.close()